This notebook will be a recreation of the study `Li, H., Cao, Y., Li, S., Zhao, J., & Sun, Y. (2020). XGBoost model and its application to personal credit evaluation. IEEE Intelligent Systems, 35(3), 52-61.`. The study originally uses dataset from Lending Club, but this source is not available anymore. Though, there copies available on kaggle. The dataset can be found [here](https://www.kaggle.com/datasets/wordsforthewise/lending-club/data).

Authors:
- Adam Janczyszyn
- Hubert Wojewoda
- Jakub Wujec
- Jakub Żmujdzin

### Data Loading

In [1]:
import pandas as pd
df = pd.read_csv('data/credit_card_2018.csv')

C:\Users\jzmuj\AppData\Local\Temp\ipykernel_9388\3524197630.py:2: DtypeWarning: Columns (129,130,131,134,135,136,139,145,146,147) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('data/credit_card_2018.csv')


Let us preprocess the data according to the study's methodology.
We start off by mapping the target variable to 0 and 1. Then, we run sampling for each month - we take 4 times as many good clients as bad clients.

In [2]:
from rr_project.data_wrangling import preprocess_data
preprocessed_df = preprocess_data(df)

C:\Users\jzmuj\Desktop\uczelnia\RR_2\Reproducible-Research-uni-project\rr_project\data_wrangling.py:22: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df["issue_d"] = pd.to_datetime(df.issue_d)


Source data shape: (127702, 153). 
Source data target distribution:
target
0    124626
1      3076
Name: count, dtype: int64
Bads for month 1: 420. Goods for month 1: 1680.
Bads for month 2: 323. Goods for month 2: 1292.
Bads for month 3: 390. Goods for month 3: 1560.
Bads for month 4: 394. Goods for month 4: 1576.
Bads for month 5: 361. Goods for month 5: 1444.
Bads for month 6: 310. Goods for month 6: 1240.
Bads for month 7: 271. Goods for month 7: 1084.
Bads for month 8: 213. Goods for month 8: 852.
Bads for month 9: 136. Goods for month 9: 544.
Bads for month 10: 133. Goods for month 10: 532.
Bads for month 11: 80. Goods for month 11: 320.
Bads for month 12: 45. Goods for month 12: 180.
Undersampled data shape: (15380, 153). 
Undersampled data target distribution:
target
0    12304
1     3076
Name: count, dtype: int64


At this first step, we already have discrepancies to the original study:
- though the number of rows is the same, there are 10 additional columns (153 vs 143). They are mainly related to metadata (e.g. url of download) and should not be used in the analysis.
- There is a difference in the number of "bad" clients. The authors used the following mapping: ["Fully Paid", "Current", "In Grace Period"] for good clients, all other for bad clients. Though, the authors obtain over 5k defaults, while we have only a bit over 3k. Due to that, the overall sampled row number is decrased from ~25k to ~15k.

Let us run feature selection for preprocessing. The authors have listed over 30 columns to drop by "manual selection". Furthermore, they get rid of all features with NaN proportion over 0.5. We will follow this approach.

In [3]:
from rr_project.data_wrangling import preprocessing_feature_selection
feature_selected_df = preprocessing_feature_selection(preprocessed_df)
print("Feature selected data shape: ", feature_selected_df.shape)

Features with over 50% missing values = 37. Index(['member_id', 'desc', 'orig_projected_additional_accrued_interest',
       'hardship_type', 'hardship_last_payment_amount',
       'hardship_payoff_balance_amount', 'hardship_loan_status',
       'hardship_dpd', 'hardship_length', 'hardship_amount', 'deferral_term',
       'hardship_status', 'hardship_reason', 'settlement_percentage',
       'settlement_amount', 'settlement_status', 'settlement_term',
       'sec_app_mths_since_last_major_derog', 'verification_status_joint',
       'sec_app_revol_util', 'sec_app_collections_12_mths_ex_med',
       'sec_app_open_acc', 'annual_inc_joint', 'dti_joint',
       'sec_app_chargeoff_within_12_mths', 'sec_app_num_rev_accts',
       'sec_app_open_act_il', 'sec_app_fico_range_high', 'sec_app_mort_acc',
       'sec_app_inq_last_6mths', 'sec_app_fico_range_low', 'revol_bal_joint',
       'mths_since_last_record', 'mths_since_recent_bc_dlq',
       'mths_since_last_major_derog', 'mths_since_recent_re

In the next steps, the authors mention label encoding. There was no mention of a particular method, only `"The text data has been first converted to numeric data that can be recognized by the computer"`. We have decided to go with `LabelEncoder()` from `sklearn.preprocessing`, which encodes labels to numbers. We will additionally display the encodings. Null values for those columns will be filled with 0.

In [4]:
from rr_project.data_wrangling import label_encode_all
label_encoded_df = label_encode_all(feature_selected_df)
print("Label encoded data shape: ", label_encoded_df.shape)

Encoding for column 'term': {' 36 months': 0, ' 60 months': 1}
Encoding for column 'grade': {'A': 0, 'B': 1, 'C': 2, 'D': 3, 'E': 4, 'F': 5, 'G': 6}
Encoding for column 'emp_length': {'0': 0, '1 year': 1, '10+ years': 2, '2 years': 3, '3 years': 4, '4 years': 5, '5 years': 6, '6 years': 7, '7 years': 8, '8 years': 9, '9 years': 10, '< 1 year': 11}
Encoding for column 'home_ownership': {'ANY': 0, 'MORTGAGE': 1, 'OWN': 2, 'RENT': 3}
Encoding for column 'verification_status': {'Not Verified': 0, 'Source Verified': 1, 'Verified': 2}
Encoding for column 'pymnt_plan': {'n': 0, 'y': 1}
Encoding for column 'title': {'Credit card refinancing': 0}
Encoding for column 'application_type': {'Individual': 0, 'Joint App': 1}
Encoding for column 'hardship_flag': {'N': 0, 'Y': 1}
Encoding for column 'disbursement_method': {'Cash': 0, 'DirectPay': 1}
Encoding for column 'debt_settlement_flag': {'N': 0, 'Y': 1}
Label encoded data shape:  (15380, 84)


The next step is to fill null values with the mean of the column.

In [5]:
from rr_project.data_wrangling import fill_nulls_with_mean
wrangled_df = fill_nulls_with_mean(label_encoded_df)
print("Filled nulls data shape: ", wrangled_df.shape)

Filled nulls data shape:  (15380, 84)


Finally, we will perform selection by Information Value, as per the study. The idea behind IV is to bin a continuous variable and calculate the WoE (Weight of Evidence) for each bin. The IV is then calculated as a sum of the differences between the WoE and the natural logarithm of the proportion of good clients to bad clients. The IV is a measure of the predictive power of a variable. The authors have selected variables with IV over 0.02. The issue is that they have not provided us with the information about binning. We have decided to use decile binning for variables which have more than 10 distinct values. Otherwise, each distinct value will be treated as a separate bin.

In [7]:
from rr_project.data_wrangling import iv_selection
df_iv_selection = iv_selection(wrangled_df, "target")
print("IV selected data shape: ", df_iv_selection.shape)

C:\Users\jzmuj\Desktop\uczelnia\RR_2\Reproducible-Research-uni-project\rr_project\data_wrangling.py:206: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  d = d0.groupby("x", as_index=False).agg({"y": ["count", "sum"]})
C:\Users\jzmuj\Desktop\uczelnia\RR_2\Reproducible-Research-uni-project\rr_project\data_wrangling.py:206: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  d = d0.groupby("x", as_index=False).agg({"y": ["count", "sum"]})
C:\Users\jzmuj\Desktop\uczelnia\RR_2\Reproducible-Research-uni-project\rr_project\data_wrangling.py:206: FutureWarning: The default of observed=False is deprecated and will be changed 

                Variable        IV
0               int_rate  0.533063
0                  grade  0.492798
0   debt_settlement_flag  0.143604
0         fico_range_low  0.118618
0        fico_range_high  0.118618
..                   ...       ...
0     num_tl_90g_dpd_24m  0.000000
0                  title  0.000000
0       num_tl_120dpd_2m  0.000000
0            policy_code  0.000000
0                  month  0.000000

[83 rows x 2 columns]
Variables with IV lower than threshold:  ['loan_amnt', 'funded_amnt', 'funded_amnt_inv', 'annual_inc', 'title', 'dti', 'delinq_2yrs', 'open_acc', 'pub_rec', 'revol_bal', 'revol_util', 'total_acc', 'collections_12_mths_ex_med', 'policy_code', 'application_type', 'acc_now_delinq', 'tot_coll_amt', 'open_act_il', 'total_bal_il', 'all_util', 'total_cu_tl', 'bc_util', 'chargeoff_within_12_mths', 'delinq_amnt', 'mort_acc', 'num_accts_ever_120_pd', 'num_actv_bc_tl', 'num_actv_rev_tl', 'num_bc_sats', 'num_bc_tl', 'num_il_tl', 'num_op_rev_tl', 'num_rev_accts', 

C:\Users\jzmuj\Desktop\uczelnia\RR_2\Reproducible-Research-uni-project\rr_project\data_wrangling.py:206: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  d = d0.groupby("x", as_index=False).agg({"y": ["count", "sum"]})
C:\Users\jzmuj\Desktop\uczelnia\RR_2\Reproducible-Research-uni-project\rr_project\data_wrangling.py:206: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  d = d0.groupby("x", as_index=False).agg({"y": ["count", "sum"]})
C:\Users\jzmuj\Desktop\uczelnia\RR_2\Reproducible-Research-uni-project\rr_project\data_wrangling.py:206: FutureWarning: The default of observed=False is deprecated and will be changed 

After those preprocessing steps, we have 39 columns left (vs. 27 in the study). Now, we will run corelation analysis

### TODO: Hubi -> The features with lower IV value for two features with the correlation coefficient greater than 0.6 were removed